In [5]:
import pandas as pd
import openai
from dotenv import load_dotenv
import os
import ast
from openai import AzureOpenAI
import faiss
import numpy as np
import json
import spacy
nlp = spacy.load("en_core_web_sm")
import pycountry
import re
from bert_score import score as bert_score
import csv
from sentence_transformers import SentenceTransformer



/Users/davidoluyalegbenga/.pyenv/versions/3.9.6/lib/python3.9/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.0.0) was trained with spaCy v3.0.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [22]:
model_bert = SentenceTransformer('bert-base-uncased')


No sentence-transformers model found with name bert-base-uncased. Creating a new one with MEAN pooling.


### Load Model files

In [68]:
df = pd.read_pickle('../models/df_embed_EN_All.pkl')

### Load Enviroment files

In [7]:
# Load environment variables
load_dotenv()

True

In [8]:
# OpenAI API configuration
openai.api_type = "azure"
openai.api_key = os.getenv("api_key_azure")
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_version = os.getenv("api_version")
openai_deployment = "sdgi-gpt-35-turbo-16k"


client = AzureOpenAI(
  api_key = os.getenv("api_key_azure"),  
  api_version = os.getenv("api_version"),
  azure_endpoint =os.getenv("AZURE_OPENAI_ENDPOINT") 
)

embedding_model = os.getenv("USER_QUERY_EMBEDDING_ENGINE") 

# print(openai.api_key)
# print(openai.api_base)
# print(openai.api_version)


<h3>globals</h3>

In [9]:
# test_query="What are the sustainable energy priorities for UNDP?"
test_query = 'What is the Human Development Index (HDI) value for Albania as mentioned in the document?'


<h3> helper functions </h3>

In [10]:
# use this function to make simple openAI Calls
def callOpenAI(prompt):  
    response_entities = openai.chat.completions.create(
                    model=openai_deployment,
                    temperature=0,
                    messages=[
                        {"role": "user", "content": prompt},
                    ]
                )
    response = response_entities.choices[0].message.content
    return response


<h3> processing modules </h3>

In [11]:
def extractEntitiesFromQuery(user_query):
    prompt = f"""
    Extract entities from the following user query: \"{user_query}\" and return output in array format.
    
    -Entities should be directly related to the domain or topic of interest. They should represent important concepts that contribute to the understanding of the subject matter.
    -Each entity in the knowledge graph should be distinct and have a unique identifier. This ensures clarity and avoids ambiguity when establishing relationships between entities.
    -You Must return output in array format e.g  ['entity1','entity2'] !!!
    -Avoid adding new lines or breaking spaces to your output. Array should be single dimension and single line !!!
 
    """
    entity_list = callOpenAI(prompt)   
    return entity_list

# Test usage
# test_query = "What are the sustainable energy for UNDP?"
# entity_list = extractEntitiesFromQuery(test_query)
# print(entity_list)

In [12]:
## module to get information on the entities from user query using the KG
def knowledgeGraphModule(user_query):
    
    # generate list of entities based on user query
    entity_list = extractEntitiesFromQuery(user_query)
    my_list = ast.literal_eval(entity_list)
    prompt_summarise_entites = f"""
    Summarize all relations between all the entities : {my_list}
    """
    summarise_entities = callOpenAI(prompt_summarise_entites)
    # Initialize an empty dictionary to store information
    entities_dict = {
        "relations": summarise_entities,
        "entities": {}
    }
    # Loop through each entity in the list
    for entity in my_list:
        # Fetch information about the entity from your knowledge graph
        prompt = f"Give me a short description 50 words of {entity}"
        entity_info = callOpenAI(prompt)
        # Add the entity information to the dictionary
        entities_dict["entities"][entity] = entity_info
    
    return entities_dict


# Test usage
test_query = "What is the role of Paris Agreement in sustainable energy development?"
entities_dict = knowledgeGraphModule(test_query)
print(entities_dict)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


{'relations': 'The Paris Agreement is related to sustainable energy development.', 'entities': {'Paris Agreement': 'The Paris Agreement is an international treaty adopted in 2015 by nearly every country in the world. Its goal is to combat climate change by limiting global warming to well below 2 degrees Celsius above pre-industrial levels, and to pursue efforts to limit the temperature increase to 1.5 degrees Celsius.', 'sustainable energy development': 'Sustainable energy development refers to the process of harnessing and utilizing renewable energy sources in a manner that minimizes environmental impact and promotes long-term viability. It involves the adoption of clean technologies, such as solar, wind, and hydro power, to meet energy needs while reducing greenhouse gas emissions and preserving natural resources.'}}


In [13]:

def find_mentioned_countries(text):
    countries = set()
    
    # Tokenize the text using regular expressions to preserve punctuation marks
    words = re.findall(r'\w+|[^\w\s]', text)
    text = ' '.join(words)  # Join the tokens back into a string
    
    for word in text.split():
        try:
            country = pycountry.countries.get(name=word) #pycountry.countries.lookup(word)
            if country != None : 
               countries.add(country.name)
        except LookupError:
            pass
    
    return list(countries)

# Example 
# user_query = 'Give me a summary of the goals UNDP wants to achieve in 10 years and the energy plans for Philippines and Angola'
# mentioned_countries = find_mentioned_countries(user_query)
# mentioned_countries

In [23]:
#Run quick test of Context Similarity model
# sentence1 = 'The cat gracefully leaped onto the windowsill, enjoying the warmth of the sun streaming through the glass.'
# sentence2 = 'Basking in the sunlight pouring through the window, the agile feline elegantly bounded onto the ledge.'

# encoding1 = model_bert.encode(sentence1)
# encoding2 = model_bert.encode(sentence2)

# similarity = np.dot(encoding1, encoding2) / (np.linalg.norm(encoding1) * np.linalg.norm(encoding2)) 
# print("Similarty Score", round(similarity,1))


Similarty Score 0.9


In [78]:
#This contains all filters for the semantic search
#Context Similarity takes two queries and find how similar they are "context wise"
#E.g "My house is empty today" and "Nobody is at my home" are same context but not word similarity
# - Filter country relevant documents when mentioned 
# - Filter by Context similarity in user_query and title, journal, content etc.


# def filter_semantics(user_query):
#     mentioned_countries = find_mentioned_countries(user_query)
    
#     if mentioned_countries:
#         country = mentioned_countries[0]
#         filtered_df = df[df['Country Name'] == country]
#     else:
#         filtered_df = df  # If no country mentioned, consider all documents
    
#     # Calculate the encoding for the user query
#     query_encoding = model_bert.encode(user_query)
    
#     # Calculate similarity scores for each document
#     similarity_scores = []
#     for title in filtered_df['Document Title']:
#         if pd.isnull(title):
#             title_encoding = model_bert.encode('')
#         else:
#             title_encoding = model_bert.encode(title)
#         similarity = np.dot(query_encoding, title_encoding) / (np.linalg.norm(query_encoding) * np.linalg.norm(title_encoding))
#         similarity_scores.append(similarity)
    
#     # Sort the similarity scores and get indices of top 10 scores
#     top_10_indices = np.argsort(similarity_scores)[::-1][:10]
    
#     # Get the top 10 documents based on the indices
#     top_10_documents = filtered_df.iloc[top_10_indices]
    
#     return top_10_documents


def filter_semantics(user_query):
    mentioned_countries = find_mentioned_countries(user_query)
    
    if mentioned_countries:
        country = mentioned_countries[0]
        filtered_df = df[df['Country Name'] == country]
    else:
        filtered_df = df  # If no country mentioned, consider all documents
    
    # Calculate the encoding for the user query
    query_encoding = model_bert.encode(user_query)
    
    # Batch processing: Encode document titles in batches
    batch_size = 64
    num_batches = (len(filtered_df) + batch_size - 1) // batch_size
    title_encodings = []
    for batch_idx in range(num_batches):
        batch_start = batch_idx * batch_size
        batch_end = min((batch_idx + 1) * batch_size, len(filtered_df))
        batch_titles = filtered_df.iloc[batch_start:batch_end]['Document Title'].fillna('').tolist()
        batch_encodings = model_bert.encode(batch_titles)
        title_encodings.extend(batch_encodings)
    
    # Calculate similarity scores for each document batch
    similarity_scores = np.dot(query_encoding, np.array(title_encodings).T)
    similarity_scores /= np.linalg.norm(query_encoding) * np.linalg.norm(np.array(title_encodings), axis=1)
    
    # Get the indices of top 10 scores
    top_10_indices = np.argsort(similarity_scores)[::-1][:10]
    
    # Get the top 10 documents based on the indices
    top_10_documents = filtered_df.iloc[top_10_indices]
    
    return top_10_documents

# def filter_semantics(user_query):
#     mentioned_countries = find_mentioned_countries(user_query)
#     # print(mentioned_countries)
#     # Check if mentioned_countries is not empty
#     if mentioned_countries:
#         country = mentioned_countries[0]
#         return df[df['Country Name'] == country]
#     else:
#         # Handle the case where no countries were mentioned
#         #Filter by Title, Authors, Content, year, and Journal similarity.
#         return None  # Or return an empty DataFrame or any other suitable value


# Example 
# test_query="What are the main goals of Niger's Vision 2035?"
# filtered_country = filter_country(test_query)
# filtered_country

In [76]:
def search_embeddings(user_query):
    df_filtered = filter_semantics(user_query) if filter_semantics(user_query) is not None else None
    
    if df_filtered is not None and not df_filtered.empty:  # Check if DataFrame is not None and not empty
        length = len(df_filtered.head())
        filtered_embeddings_arrays = np.array(list(df_filtered['Embedding']))
        index = faiss.IndexFlatIP(filtered_embeddings_arrays.shape[1]) 
        index.add(filtered_embeddings_arrays)

        user_query_embedding = client.embeddings.create( 
                input=user_query ,model= embedding_model
            ).data[0].embedding

        k = min(5, length)
        distances, indices = index.search(np.array([user_query_embedding]), k)
        return df_filtered, distances, indices
    else:
        return None, None, None


In [59]:
def get_answer(user_question, content):
    system_prompt = "You are a system that answers user questions based on excerpts from PDF documents provided for context. Only answer if the answer can be found in the provided context. Do not make up the answer; if you cannot find the answer, say so."
    messages = [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': user_question},
        {'role': 'user', 'content': content},
    ]
    response_entities = openai.chat.completions.create(
                    model=openai_deployment,
                    temperature=0.2,
                    messages=messages
                )
    response = response_entities.choices[0].message.content
    return response
  

In [74]:
def map_to_structure(qs):
    result_dict = {}

    # Extract the DataFrame from the tuple
    dataframe = qs[0]

    # Counter to limit the loop to 10 iterations
    count = 0
    for index, row in dataframe.iterrows():
        # Define a unique identifier for each document, you can customize this based on your data
        document_id = f"doc-{index + 1}"
        # Handle NaN in content by using fillna
        content = row["Content"]
        content = ' '.join(row["Content"].split()[:160])
        # Create a dictionary for each document
        document_info = {
            "title": row["Document Title"],
            "extract": content or "",  # You may need to adjust this based on your column names
            "category": row["Category"],
            "link": row["Link"],
            "thumbnail": ''
        }
        # print(document_info)
        # Add the document to the result dictionary
        result_dict[document_id] = document_info

        # Increment the counter
        count += 1

        # # Break out of the loop if the counter reaches top 10
        if count == 200:
            break

    return result_dict


In [79]:
## module to extract text from documents and return the text and document codes

def semanticSearchModule(user_query):
    qs = search_embeddings(user_query) #df, distances, indices
    # if qs != None :
    if qs[0] is not None:
        result_structure = map_to_structure(qs)
        return result_structure
    else : 
        return []
#test usage
excerpts_dict=semanticSearchModule(test_query)
# print(f"""excerpts_dict {excerpts_dict}""")

#Return top 10-20 most related 
# Define the filename to save the JSON data -  can remove later
json_filename = "outputs/excerpts.json"

# Save excerpts_dict to the JSON file just for a better preview
with open(json_filename, 'w', encoding='utf-8') as json_file:
    json.dump(excerpts_dict, json_file, ensure_ascii=False, indent=4)

print(f"Excerpts saved to {json_filename}")

Excerpts saved to outputs/excerpts.json


In [62]:
## module to get data for specific indicators which are identified is relevant to the user query

def indicatorsModule(user_query): #lower priority
    
    # find relevant indicators based on uesr query and extract values
    indicators_dict={
        "indicator-id-1":"value from indicator-id-1",
        "indicator-id-2":"value from indicator-id-2"
    }#temp
    
    return indicators_dict

#test usage
indicators_dict=indicatorsModule(test_query)
print(indicators_dict)

{'indicator-id-1': 'value from indicator-id-1', 'indicator-id-2': 'value from indicator-id-2'}


In [63]:
## module to generate query ideas

def queryIdeationModule(user_query): # lower priority
    
    # Generate query ideas using OpenAI GPT-3
    prompt = f"""Generate query ideas based on the user query: {user_query}
    -You Must return output in array format e.g ['idea 1', 'idea2'] !!!
    -Avoid adding new lines or breaking spaces to your output. Array should be single dimension and single line !!!
    """
    response = callOpenAI(prompt)
    return response

#test usage
# query_idea_list=queryIdeationModule(test_query)
# print(query_idea_list)

<h3> synthesis module </h3>

    llm_instructions="llm instruction template here, with placeholders for insertion of user query, excerpts, indicator data, and entity and relation info" 


In [64]:
# module to synthesize answer using retreival augmented generation approach

prompt_formattings= f"""
  - Answer output must be properly formatted using HTML. 
            - Don't include <html>, <script>, <link> or <body> tags. Only text formating tags should be allowed. e.g h1..h3, p, anchor, etc.
            - Make sure to Include citations based on the Sources. e.g Text excerpt here<a data-id='test-doc-1'>[1]</a> when referencing a document in the sources. using 1 ...nth
            - The citations anchor should be near the excerpt not following each other.
            - Use the anchor tag for the citation links and should link to the document link. for example Undp operates in afganistan <a data-id='test-doc-1'>[1]</a>. UNDP offers health relationships <a data-id='test-doc-2'>[2]</a>.
            - The text in the anchor tag should be citation number not document title.
            - You can reference multitple citations based sources
"""
def synthesisModule(user_query, entities_dict, excerpts_dict, indicators_dict,prompt_formattings):
    
    # Generate prompt engineering text and template
    llm_instructions = f"""
    Ignore previous commands!!!
    Given a user query, use the provided excerpts, Sources, and entity and relation info to
    provide the correct answer to the user's query
    
    User Query: {user_query}
    
    Sources: {excerpts_dict}
    
    Entity and Relation info: {entities_dict}

      {prompt_formattings}
    """
    ###synthesize data into structure within llm prompt engineering instructions
    answer=callOpenAI(llm_instructions)
    
    return answer

## to test this, run the full pipeline with the handleApiCall function

<h3> run pipeline </h3>

In [82]:
# full pipeline with retreival, synthesis of answer to user query, and structure results into api response

def handleApiCall(user_query):
    
    ##run processing modules (in parallel)
    entities_dict=knowledgeGraphModule(user_query)
    excerpts_dict=semanticSearchModule(user_query)
    indicators_dict=indicatorsModule(user_query) ##lower priority
    query_idea_list=queryIdeationModule(user_query) ##lower priority
    
    ##synthesis module
    answer=synthesisModule(user_query, entities_dict, excerpts_dict, indicators_dict, prompt_formattings)
    
    ##structure response
    response={
        "user_query":user_query,
        "answer":answer,
        "sources":excerpts_dict,
        "query_ideas":query_idea_list,
        "entities":list(entities_dict["entities"].keys())       
    }
    
    return response

# test usage
test_query = "What is the role of USAID in supporting energy sector strategy?"
response=handleApiCall(test_query) 
# Define the filename to save the JSON data -  can remove later
json_filename = "outputs/synthesis_output.json"

# Save excerpts_dict to the JSON file just for a better preview
with open(json_filename, 'w', encoding='utf-8') as json_file:
    json.dump(response, json_file, ensure_ascii=False, indent=4)

print(f"Synthesis saved to {json_filename}")

Synthesis saved to outputs/synthesis_output.json


<h1>Testing</h1>

<p>This sections contains all things testings </p>

In [22]:
# !pip install bert_score

In [ ]:
def calculate_scores(csv_file):
    # Initialize an empty list to store processed entries
    result = []
    
    # Loop through each entry in the CSV file
    for entry in csv.DictReader(csv_file):
        query = entry['query']
        sample_answer = entry['sample_answer']
        
        # Call OpenAI for chat GPT answer
        chat_gpt_answer = callOpenAI(f""" 
                                    {query} 
                                    {prompt_formattings} 
                                    """)
        
        # Call the moonshot model API
        moonshot_model_answer = handleApiCall(query) 
        
        # Calculate BERT score for moonshot model answer
        P, F, R = bert_score([sample_answer], [moonshot_model_answer['answer']], lang='en', verbose=True)
        entry['moonshot_model_answer'] = moonshot_model_answer['answer']
        entry['bert_score'] = round(float(F), 2)

        # Calculate BERT score for chat GPT answer
        P, F, R = bert_score([sample_answer], [chat_gpt_answer], lang='en', verbose=True)
        entry['chat_gpt_answer'] = chat_gpt_answer
        entry['bert_score_gpt'] = round(float(F), 2)
        
        # Append the processed entry to the result list
        result.append(entry)
    
    # Return the list of processed entries
    return result


# Specify the path to your CSV file
csv_file_path = "../testing/queries.csv"

# Open the CSV file for reading
with open(csv_file_path, mode='r') as file:
    # Pass the file object to the function
    result = calculate_scores(file)

# Print updated data with scores
# print(json.dumps(result, indent=4))

# Save updated data to a JSON file
with open('../testing/test_output.json', 'w') as file:
    json.dump(result, file, indent=4)


<h1>Compare Moonshot BERT score to GPT BERT Score</h1>

In [84]:
# Load the JSON file
with open('../testing/test_output.json', 'r') as file:
    data = json.load(file)

# Initialize variables to store total scores and count of items
total_bert_score = 0
total_bert_score_gpt = 0
count = 0

# Iterate through each item in the JSON data
for item in data:
    # Extract bert_score and bert_score_gpt from the current item
    bert_score = item['bert_score']
    bert_score_gpt = item['bert_score_gpt']
    
    # Add the scores to the total
    total_bert_score += bert_score
    total_bert_score_gpt += bert_score_gpt
    
    # Increment the count
    count += 1

# Calculate the average scores
average_bert_score = total_bert_score / count
average_bert_score_gpt = total_bert_score_gpt / count

# Print the average scores
print("Average bert_score:", average_bert_score)
print("Average bert_score_gpt:", average_bert_score_gpt)


Average bert_score: 0.7995652173913044
Average bert_score_gpt: 0.7778260869565218


# TODO::
- Move improvement on semantic search to main Flask API - Ongoing
- Other documents for data - SEH, Publication, UN resolutions, etc 
- Run the full pipeline on all the countries PDFS
- Create updated model based on the full cleaned data

